In [1]:
import import_ipynb
from news_classes import *
import math
from textblob import TextBlob as tb
import os
import pickle
from collections import Counter
from textblob import TextBlob as tb
import numpy as np

importing Jupyter notebook from news_classes.ipynb


In [2]:
idf_path='idf_30_9_2018.pkl'
with open(idf_path, 'rb') as input:
    idfblob = pickle.load(input)

In [3]:
def idf(word):
    return math.log(idfblob.size() / (1 + idfblob.getIdf(word)))

In [4]:
def sent_cosine(sentence1, sentence2):
    count_sent1=Counter(sentence1.words)
    count_sent2=Counter(sentence2.words)
    
    up=0
    downright=0
    downleft=0
    
    words=set()
    for word in count_sent1.keys():
        words.add(word)
    for word in count_sent2.keys():
        words.add(word)
    
    for word in words:
        cnt1=count_sent1.get(word)
        cnt2=count_sent2.get(word)
        if cnt1 is None:
            cnt1=0            
        elif cnt2 is None:
            cnt2=0
                
        up+=cnt1*cnt2*idf(word)*idf(word)
        downright+=cnt1*idf(word)*cnt1*idf(word)
        downleft+=cnt2*idf(word)*cnt2*idf(word)
        
    try:
        return up/(math.sqrt(downleft)*math.sqrt(downright))
    except:
        return 0

In [5]:
#w,h=len(sentences), len(sentences)
def sent_to_matrix(sentences):
    w,h=len(sentences), len(sentences)
    dist_matrix = np.zeros(shape=(w,h))
    for i in range(w):
        for j in range(h):
            dist = sent_cosine(sentences[i],sentences[j])
            dist_matrix[i][j]=dist
            dist_matrix[j][i]=dist
    return dist_matrix

In [31]:
def get_top_ranked_sentences(sentences, threshold, number_of_summarized_sentences):
    lower_sentences=[]
    for sentence in sentences:
        lower_sentences.append(sentence.lower())
    matrix = sent_to_matrix(lower_sentences)
    
    degrees=np.zeros(shape=(len(matrix)))
    for j in range(len(matrix)):
        degrees[j]=sum(i > threshold for i in matrix[j])
    
    indices=[]
    for i in range(len(degrees)):
        indices.append(i)
        
    sentences = [x for _, x in sorted(zip(degrees,sentences), reverse=True)]
    indices = [x for _, x in sorted(zip(degrees,indices), reverse=True)]
    
    new_sentences = []
    new_indices = []
    
    for i in range(number_of_summarized_sentences):
        new_sentences.append(sentences[i])
        new_indices.append(indices[i])
    
    new_sentences = [x for _, x in sorted(zip(new_indices,new_sentences))]
    return new_sentences

In [32]:
def summarize(para, number_of_summarized_sentences):
    sentences = tb(para).sentences
    sentences=get_top_ranked_sentences(sentences, .1, number_of_summarized_sentences)
    
    summ=""
    for sentence in sentences:
        summ+=str(sentence)
        
    return summ

In [33]:
para = "Education helps in the mental and intellectual nourishment and growth of a person. Without education an individual cannot progress intellectually, and cannot develop skills and capacities to work. Education fosters the enlightenment, empowerment and emancipation of society. Without education a society will be socially, economically and politically backward. Education is what serves to make a nation strong, capable, powerful and resilient, and take it forward. Education is, therefore, very important for every individual person, and as a whole for every society and nation. Hence, education must be available to all the children and youth of a nation without any discrimination."

In [34]:
summ = summarize(para, 3)

In [30]:
print(para)
print(summ)

Education helps in the mental and intellectual nourishment and growth of a person. Without education an individual cannot progress intellectually, and cannot develop skills and capacities to work. Education fosters the enlightenment, empowerment and emancipation of society. Without education a society will be socially, economically and politically backward. Education is what serves to make a nation strong, capable, powerful and resilient, and take it forward. Education is, therefore, very important for every individual person, and as a whole for every society and nation. Hence, education must be available to all the children and youth of a nation without any discrimination.
Without education a society will be socially, economically and politically backward.Education is, therefore, very important for every individual person, and as a whole for every society and nation.Hence, education must be available to all the children and youth of a nation without any discrimination.


In [11]:
X = ["a","b","c","d","e"]
Y = [2,4,10,6,0]

In [12]:
X = [x for _, x in sorted(zip(Y,X))]

In [13]:
print(X)
print(Y)

['e', 'a', 'b', 'd', 'c']
[2, 4, 10, 6, 0]
